In [11]:
#imports
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import os 
from sklearn.preprocessing import MinMaxScaler,StandardScaler


ModuleNotFoundError: No module named 'tensoflow'

In [18]:
#loading dataset using panda 
#this loads one specific csv file
#dataset = pd.read_csv(r'C:\Users\benja\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data\ETFs\aadr.us.txt')
#dataset.head()
#recursively load all csv files into a list
#sets my working directory so the glob can fetch all the files
os.chdir(r'C:\Users\Admin\OneDrive\Dokumente\Wichtiges\Projects\stock_price_predictor\dataset\archive\Data')
file_list = glob.glob('**/*.*',recursive=True)
"""""
dataset = pd.read_csv(file_list[0],delimiter=',',header=0)
print(dataset.info)
dataset2 = pd.read_csv(file_list[1],delimiter=',',header=0)
print(dataset2.info)
result = pd.concat([dataset.reset_index(drop=True),dataset2.reset_index(drop=True)])
print(result)
"""
dataset = pd.DataFrame()
#merges every dataframe in one big dataframe
"""""
for file in file_list:
    if(os.stat(file).st_size!=0):
        temp = pd.read_csv(file,sep=',',header=0)
        dataset = pd.concat([dataset.reset_index(drop=True),temp.reset_index(drop=True)])
print(dataset)
"""
#merges every dataframe in one big dataframe and check for empty files
for file in file_list:
    if(os.stat(file).st_size!=0):
        temp = pd.read_csv(file,sep=',',header=0)
        dataset = pd.concat([dataset.reset_index(drop=True),temp.reset_index(drop=True)])


In [20]:
dataset.head()

,Date,Open,High,Low,Close,Volume,OpenInt
0,2010-07-21,24.333,24.333,23.946,23.946,43321,0
1,2010-07-22,24.644,24.644,24.362,24.487,18031,0
2,2010-07-23,24.759,24.759,24.314,24.507,8897,0
3,2010-07-26,24.624,24.624,24.449,24.595,19443,0
4,2010-07-27,24.477,24.517,24.431,24.517,8456,0


Data Preprocessing

In [21]:

#order the dataset by date --> don't know if this is necessary
dataset = dataset.sort_values(by=['Date'])
#droping the date column --> not needed
dataset = dataset.drop(['Date'],axis=1)
#rescale the data to -1-1
scaler = MinMaxScaler(feature_range=(-1,1))
dataset = scaler.fit_transform(dataset)


In [10]:
#visualize the data

%load_ext tensorboard

Splitting data into training, test, validation set

In [25]:
#function for splitting the data since the predicted label is always the stock price for of the "next day"
def split(dataset, n):
    x,y = [],[]
    for i in range(len(dataset)-n+1):
        x.append(dataset[i:i+n,:-1])
        y.append(dataset[i+n-1,-1])
    return np.array(x),np.array(y) 
def split_sets(x,y,x_split):
    x_train = x[:x_split]
    y_train = y[:x_split]
    x_test = x[x_split:]
    y_test = y[x_split:]
    return x_train,y_train,x_test,y_test

In [24]:
x,y = split(dataset,2)
#define training split --> 80% training, 10% testing, 10% validation
training_split = 0.8
#get the index where the data should be splitted
x_split = int(np.ceil(len(x)*training_split))
#split the data into training and testing and validation sets
x_train,y_train,x_test,y_test = split_sets(x,y,x_split)
#split the testing data into testing and validation sets with 70% testing and 30% validation
testing_split = 0.7
x_split = int(np.ceil(len(x_test)*testing_split))
x_test,y_test,x_val,y_val = split_sets(x_test,y_test,x_split)
